In [0]:
import pandas as pd
import numpy as np

# Data frame for training data

ID      target  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  \
0  000d6aaf2  38000000.0        0.0          0        0.0          0   
1  000fbd867    600000.0        0.0          0        0.0          0   
2  0027d6b71  10000000.0        0.0          0        0.0          0   
3  0028cbf45   2000000.0        0.0          0        0.0          0   
4  002a68644  14400000.0        0.0          0        0.0          0   

   2f0771a37  30347e683  d08d1fbe3  6ee66e115    ...      3ecc09859  \
0          0          0          0          0    ...            0.0   
1          0          0          0          0    ...            0.0   
2          0          0          0          0    ...            0.0   
3          0          0          0          0    ...            0.0   
4          0          0          0          0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0          0          0          0          0   
1        0.0        0.0          0          0          0          0   
2        0.0        0.0          0          0          0          0   
3        0.0        0.0          0          0          0          0   
4        0.0        0.0          0          0          0          0   

   fb36b89d9  7e293fbaf  9fc776466  
0          0          0          0  
1          0          0          0  
2          0          0          0  
3          0          0          0  
4          0          0          0 

In [1]:
#!pip install pydrive

# Data and authentication
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth

# Authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

# Load data
drive = GoogleDrive(gauth)

train_downloaded = drive.CreateFile({'id': '17L5XwAyCcOseGvRSLD60LYyCcYVFaw7i'})
train_downloaded.GetContentFile('train.csv')

test_downloaded = drive.CreateFile({'id': '1AA_GltbU5s_LlRAKGJFB9QGQR7sEbgIZ'})
test_downloaded.GetContentFile('test.csv')

# Data to pandas data frames
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

NameError: ignored

In [0]:
from sklearn.model_selection import train_test_split
import xgboost as xgb


# Data frames for modeling
train = df_train.drop(['ID', 'target'], axis=1)
target = df_train['target']

test = df_test.drop(['ID'], axis=1)
tdf = pd.DataFrame()

reg = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
reg.fit(train, target)
pred = reg.predict(test)
preList = []

for prediction in pred.astype(int):
  preList.append(abs(prediction))
  
tdf['ID'] = df_test['ID']
tdf['target'] = preList

In [37]:
print('Number of IDs: %s, Number of predictions: %s, tdf DataFrame size: %s' % (len(test.index), len(tdf.index), tdf.size))

from openpyxl.workbook import Workbook
from openpyxl.writer.excel import save_virtual_workbook

auth.authenticate_user()

wb = Workbook()
writer = pd.ExcelWriter('', engine='openpyxl')
writer.book = wb
tdf.to_excel(writer)

f = drive.CreateFile({'id': '181tDc1soEVd7aLz9vM24T8dQHpVuYgyO'}) #submission.csv
f.GetContentFile('submission.csv')

with open('submission.csv', 'a') as file:
    tdf.to_csv(file, index=False, header=True)

f.SetContentFile('submission.csv')
f.Upload()

Number of IDs: 49342, Number of predictions: 49342, tdf DataFrame size: 98684
